In [132]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import Lasso
# Importar bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Explicador LIME
import lime
import lime.lime_text
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Isas_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [133]:
# Exemplo de conjunto de dados de resenhas
dados = pd.read_csv('dados.csv', sep=';')

# Separar as features e o alvo
X = dados['review']
y = dados['sentimentos']

# Escolher uma instância para fazer a previsão
instance_index = 15
instance = X.iloc[instance_index]
instance_label = y.iloc[instance_index]
print('Instância:', instance)
print('Rótulo:', instance_label)
print([np.random.randint(2, size = 10) for i in range(3)])

Instância: Nunca vi algo tão ruim na minha vida. A produção foi amadora e a história, entediante.
Rótulo: 0
[array([0, 0, 1, 0, 1, 1, 1, 1, 0, 1]), array([0, 1, 0, 1, 1, 0, 0, 1, 0, 1]), array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0])]


In [134]:
# Retirar stop-words
stop_words = stopwords.words('portuguese')
X_sw = X.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Vetorização do texto
vectorizer_sw = TfidfVectorizer()
X_vectorized_sw = vectorizer_sw.fit_transform(X_sw)

# Dividir os dados em conjuntos de treino e teste
X_train_sw, X_test_sw, y_train, y_test = train_test_split(X_vectorized_sw, y, test_size=0.2, random_state=42)

In [135]:
a=np.array([0, 1, 0, 1, 0])
print(np.where(a==1)[0])

[1 3]


In [136]:
# Deixar stopwords
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)
print(vectorizer.transform(['desperdício de tempo']).indices)
print(X[59])
print(X_vectorized[59].indices)
index = vectorizer.vocabulary_.get("desperdício")
print(index)
print(X_vectorized[59, index])
print("!"*20)
a = X_vectorized[59].indices

print(a)
print("!"*20)
print(vectorizer.get_feature_names_out())
#print(vectorizer.get_feature_names_out()[118])
#print(vectorizer.get_feature_names_out()[54])
#print(vectorizer.get_feature_names_out()[246])
#print(vectorizer.get_feature_names_out()[65])
#print(vectorizer.get_feature_names_out()[239])
#print(vectorizer.get_feature_names_out()[230])
#print(vectorizer.get_feature_names_out()[217])
#print("-"*20)
#print(X_vectorized[1][0])
#print(X[1])
print(X_vectorized[59][0])
print(vectorizer.get_feature_names_out()[175])
print(X.shape)
print(X_vectorized.shape)
# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

[ 54  65 230]
Não gostei. Muito chato e sem graça. Não vale a pena.
[163 175 124 219  40 248 182 125]
65
0.0
!!!!!!!!!!!!!!!!!!!!
[163 175 124 219  40 248 182 125]
!!!!!!!!!!!!!!!!!!!!
['absolutamente' 'achei' 'acrescentaram' 'adorei' 'ainda' 'algo'
 'altamente' 'amadora' 'amei' 'anterior' 'ao' 'aos' 'apreciam' 'arte'
 'artificiais' 'as' 'assisti' 'assistindo' 'assistir' 'atento' 'atenção'
 'ator' 'atores' 'atuação' 'atuações' 'até' 'bastante' 'bela' 'belas'
 'bem' 'bom' 'brilhante' 'cabeça' 'cada' 'cadeira' 'captura' 'cativam'
 'cativante' 'cena' 'cenas' 'chato' 'chatos' 'cheio' 'cinema'
 'cinematográfica' 'com' 'começo' 'complementa' 'conectar' 'confusa'
 'confuso' 'consegui' 'cuidadosamente' 'da' 'de' 'decepcionante' 'deixa'
 'deixou' 'deles' 'desde' 'desejar' 'desenvolvida' 'desenvolvidos'
 'desinteressados' 'desinteressante' 'desperdício' 'deste' 'detalhe'
 'dias' 'dignas' 'direção' 'dirigido' 'diálogos' 'do' 'dos' 'drama'
 'efeitos' 'elaborada' 'em' 'emocional' 'emocionante' 'emo

In [137]:
def importancia(W: np.array) -> np.array:
    I = np.zeros(W.shape[1])  # Vetor de importância
    for j in range(W.shape[1]):
        soma = np.sum(np.abs(W[:, j]))
        I[j] = np.sqrt(soma)
    return I

def c(V: list, W: np.array, I: np.array) -> float:
    c_value = 0  # Valor de c
    for j in range(W.shape[1]):
        if any(W[i, j] > 0 for i in V):  # Se a característica j é relevante
            c_value += I[j]
    return c_value

def guloso(W: np.array, I: np.array, B: int) -> list:
    nao_selecionados = list(range(W.shape[0]))  # Lista de índices não selecionados
    V = []  # Conjunto de características selecionadas
    itens = 0  # Número de elementos selecionados
    c_value = 0  # Valor de c
    
    while itens < B and nao_selecionados:
        best_gain = -np.inf  # Valor de ganho máximo
        best_item = None  # Índice do melhor item 
        
        for item in nao_selecionados:  # Itera sobre os itens não selecionados
            lista_temp = V + [item]
            gain = c(lista_temp, W, I) - c(V, W, I)
            if gain > best_gain:
                best_gain = gain  # Atualiza o valor de ganho máximo
                best_item = item  # Atualiza o melhor item
        
        if best_item is not None:
            V.append(best_item)  # Adiciona o melhor item ao conjunto
            nao_selecionados.remove(best_item)  # Remove o melhor item dos não selecionados
            itens += 1
            c_value += best_gain
    
    return V

In [138]:
# Exemplo de uso
W = np.array([[1, 2, 1], [-1, 0, 2], [-2, 2, -2]])  # Exemplo de matriz W
I = importancia(W)
print("Matriz de importância: ", I)

V = [0, 1]  # Lista de índices das instâncias selecionadas
print("Cobertura: ", c(V, W, I))

V_linha = guloso(W, I, 2)
print("Melhor conjunto: ", V_linha)

Matriz de importância:  [2.           2.           2.2360679775]
Cobertura:  6.23606797749979
Melhor conjunto:  [0, 1]


In [139]:
print(np.zeros(4))

[0. 0. 0. 0.]


In [140]:
x = X_vectorized[59]
print(X[59])
def binarize(x):
    n = len(vectorizer.get_feature_names_out())
    x_bin=np.zeros(n, dtype=int)
    for i in x.indices:
        x_bin[i] = 1
    return x_bin
num_samples = 5
n = len(vectorizer.get_feature_names_out())
x_indices = x.indices
n_x_words = len(x.indices)    
sample_set = [np.zeros(n) for i in range(num_samples-1)]
sample_set.append(binarize(x))
for i in range(num_samples-1):
    z_line_indices = np.random.randint(2, size=n_x_words)
    while not np.any(z_line_indices):  
        z_line_indices = np.random.randint(2, size=n_x_words)
    z_line_indices = np.where(z_line_indices == 1)[0]
    activated_words = [x_indices[j] for j in z_line_indices]
    sample_set[i][activated_words] = 1

indices = np.where(binarize(x)== 1)[0]
z=" ".join([vectorizer.get_feature_names_out()[indice] for indice in indices])
print(z)

Não gostei. Muito chato e sem graça. Não vale a pena.
chato gostei graça muito não pena sem vale


In [163]:
class LimeExplainerSentences:
    def __init__(self, sigma=0.2, num_samples=1000, K=5, alpha=0.1, vectorizer=None, model=None):
        self.sigma = sigma
        self.num_samples = num_samples
        self.K = K
        self.alpha = alpha
        self.model = model
        self.vectorizer = vectorizer
    
    # Binarizar vetor de palavras
    def binarize(self, x):
        n = len(vectorizer.get_feature_names_out())
        x_bin=np.zeros(n, dtype=int)
        for i in x.indices:
            x_bin[i] = 1
        return x_bin

    # Define a função de kernel 
    def kernel(self, x, z):
        distance = cosine_similarity(x, z) # Similaridade de cosseno
        weights = np.sqrt(np.exp(-(distance**2) / (self.sigma**2)))  # Kernel 
        return weights
        
    # Gera dados ao redor de x_line
    def samples(self, x):
        n = len(vectorizer.get_feature_names_out())
        x_indices = x.indices
        n_x_words = len(x.indices)    
        sample_set = [np.zeros(n) for i in range(self.num_samples-1)]
        sample_set.append(self.binarize(x))
        for i in range(self.num_samples-1):
            z_line_indices = np.random.randint(2, size=n_x_words)
            while not np.any(z_line_indices):  
                z_line_indices = np.random.randint(2, size=n_x_words)
            z_line_indices = np.where(z_line_indices == 1)[0]
            activated_words = [x_indices[j] for j in z_line_indices]
            sample_set[i][activated_words] = 1
        return sample_set
    
    # Transforma um vetor em uma frase
    def sentences_samples(self, z_line):
        indices = np.where(z_line == 1)[0]
        z=" ".join([vectorizer.get_feature_names_out()[indice] for indice in indices])
        return vectorizer.transform(z)

    # Define o vetor de pesos
    def LIME(self, x):
        Z_line = self.samples(x)
        print(Z_line)
        print(len(Z_line))
        Z=[]
        for i in range(len(Z_line)):
            Z.append(self.sentences_samples(Z_line[i]))
        Z_pred = [self.model.predict(z) for z in Z]
        pi_x = []
        for z in Z:
            pi_x.append(self.kernel(x, z)) 
        lasso = Lasso(alpha=self.alpha)
        lasso.fit(Z_line, Z_pred, sample_weight=pi_x)
        w = lasso.coef_
        return w
    
    def K_top_indices(self, x):
        abs_valores = np.abs(w)

    # Gerar explicação
    def explain_instance(self, x):
        w = self.LIME(x)
        abs_valores = np.abs(w)
        indices = np.argsort(abs_valores)[::-1][:self.K]
        print(f"Frase: {x}")
        print(f"w: {w}")
        print("Palavras importantes:")
        for i, word in enumerate(x):
            if i in indices:
                print(f"{word}: {w[i]}")    


In [155]:
# Treinamento do modelo de Gradient Boosting
gb_model_sw = GradientBoostingClassifier(random_state=42)
gb_model_sw.fit(X_train_sw, y_train)


GradientBoostingClassifier(random_state=42)

In [164]:
# Retirar stop-words
stop_words = stopwords.words('portuguese')
X_sw = X.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# Vetorização do texto
vectorizer_sw = TfidfVectorizer()
X_vectorized_sw = vectorizer_sw.fit_transform(X_sw)

# Dividir os dados em conjuntos de treino e teste
X_train_sw, X_test_sw, y_train, y_test = train_test_split(X_vectorized_sw, y, test_size=0.2, random_state=42)
# Separar as features e o alvo
instance_index = 15
instance = X.iloc[instance_index]
instance_label = y.iloc[instance_index]
x = X_vectorized[instance_index]
print("Instância: ", x.indices)
print("Instância: ", instance)
print("Rótulo: ", instance_label)

LIME = LimeExplainerSentences(sigma=0.1, num_samples=2, K=3, alpha=0.1, vectorizer=vectorizer, model=gb_model_sw)
LIME.explain_instance(x)

Instância:  [118 126 218 174 250   5 245 164 158 252 200   7  89]
Instância:  Nunca vi algo tão ruim na minha vida. A produção foi amadora e a história, entediante.
Rótulo:  0
[array([0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

ValueError: Iterable over raw text documents expected, string object received.